In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/data1.csv

'Colab Notebooks'   Driveace  'Python_Programming_Assignment (2) (1).gdoc'
 data1.csv	    GDToT


In [4]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

In [25]:
# https://www.kaggle.com/c/nlp-getting-started : NLP Disaster Tweets
df = pd.read_csv("/content/drive/MyDrive/data1.csv")

In [26]:
df.shape

(83, 3)

In [27]:
df.head()

,url,type,text
0,hhs.gov,medical,Skip to main content Find infa...
1,drugs.com,medical,Skip to main content Drugs A-Z ...
2,epocrates.com,medical,Menu Features Features Features Product Feat...
3,everydayhealth.com,medical,Menu Newsletters Search Health Conditions A-Z...
4,mayoclinic.org,medical,"COVID-19: Advice, updates an..."


In [28]:
print((df.type == 'medical').sum()) # Disaster
print((df.type == 'education').sum()) # No Disaster
print((df.type == 'construction').sum()) # No Disaster
print((df.type == 'service').sum()) # No Disaster

22
26
25
10


In [29]:
df['type'] = df['type'].map({'medical': 0, 'education': 1, 'construction': 2, 'service': 3})


In [30]:
df.head()
df.tail()

,url,type,text
78,domo.com,3,Watch Demo TRY FREE Use the ...
79,ivision.com,3,Skip to main content News Pre...
80,waveoc.com,3,You need to switch on JavaScript for...
81,shopping-cart-migration.com,3,× Try It Free Pay only for what you migrate...
82,interamark.com,3,(818) 796-3060 ...


In [31]:
# Preprocessing
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

# https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate/34294022
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [32]:
pattern = re.compile(r"https?://(\S+|www)\.\S+")
for t in df.text:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print(match)
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

       Skip navigation     National Library of Medicine          Menu   Health Topics  Drugs & Supplements  Genetics  Medical Tests  Videos & Tools  About MedlinePlus  Show Search      Search MedlinePlus     GO       About MedlinePlus  What's New  Site Map  Customer Support      Health Topics Drugs & Supplements Genetics Medical Tests Videos & Tools    Español     COVID-19 COVID.gov | NIH research information | Español | NIH staff guidance (NIH Only) x           Health Topics  Find information on health, wellness, disorders and conditions         Drugs & Supplements  Learn about prescription drugs, over-the-counter medicines, herbs, and supplements         Genetics   Explore genetic conditions, genes, and more         Medical Tests  Learn why your doctor orders medical tests and what the results may mean         Medical Encyclopedia  Articles and images for diseases, symptoms, tests, treatments         Healthy Recipes  Try these recipes as part of a healthy lifestyle        Share Medli

In [33]:
df["text"] = df.text.map(remove_URL) # map(lambda x: remove_URL(x))
df["text"] = df.text.map(remove_punct)

In [34]:
# remove stopwords
# pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine
# has been programmed to ignore, both when indexing entries for searching and when retrieving them 
# as the result of a search query.
stop = set(stopwords.words("english"))

# https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop

In [35]:
df["text"] = df.text.map(remove_stopwords)

In [36]:
df.text

0     skip main content find infant formula resource...
1     skip main content drugs az pill identifier int...
2     menu features features features product featur...
3     menu newsletters search health conditions az c...
4     covid19 advice updates vaccine options find co...
                            ...                        
78    watch demo try free use arrows select result p...
79    skip main content news press releases communit...
80    need switch javascript full functionality webs...
81    × try free pay migrate cost depends number rec...
82    818 7963060 services marketing services graphi...
Name: text, Length: 83, dtype: object

In [37]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(df.text)

In [39]:
len(counter)

12713

In [ ]:
counter

In [40]:
counter.most_common(5)

[('health', 330),
 ('us', 308),
 ('solutions', 294),
 ('new', 290),
 ('class', 268)]

In [41]:
num_unique_words = len(counter)

In [84]:
# Split dataset into training and validation set
train_size = int(df.shape[0] * 0.8)

train_df = df.sample(frac=0.8)
val_df = df.sample(frac=0.2)

# split text and labels
train_sentences = train_df.text.to_numpy()
train_labels = train_df["type"].to_numpy()
val_sentences = val_df.text.to_numpy()
val_labels = val_df["type"].to_numpy()
train_labels = train_labels/3.0001
val_labels = val_labels/3.0001
print(train_labels, val_labels)

[0.33332222 0.66664445 0.         0.99996667 0.33332222 0.99996667
 0.99996667 0.66664445 0.33332222 0.         0.66664445 0.
 0.33332222 0.33332222 0.33332222 0.         0.66664445 0.66664445
 0.66664445 0.33332222 0.66664445 0.66664445 0.         0.66664445
 0.66664445 0.33332222 0.33332222 0.         0.66664445 0.
 0.66664445 0.66664445 0.33332222 0.66664445 0.         0.33332222
 0.33332222 0.66664445 0.33332222 0.         0.99996667 0.
 0.66664445 0.99996667 0.66664445 0.66664445 0.66664445 0.
 0.99996667 0.66664445 0.         0.33332222 0.33332222 0.
 0.33332222 0.         0.66664445 0.         0.33332222 0.33332222
 0.33332222 0.33332222 0.33332222 0.33332222 0.99996667 0.66664445] [0.99996667 0.66664445 0.99996667 0.66664445 0.66664445 0.
 0.99996667 0.33332222 0.66664445 0.66664445 0.66664445 0.66664445
 0.33332222 0.66664445 0.33332222 0.33332222 0.        ]


In [64]:
train_sentences.shape, val_sentences.shape

((66,), (17,))

In [65]:
# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training

In [66]:
# each word has unique index
word_index = tokenizer.word_index

In [67]:
word_index

{'health': 1,
 'solutions': 2,
 'class': 3,
 'us': 4,
 'home': 5,
 'new': 6,
 'science': 7,
 'free': 8,
 'may': 9,
 'data': 10,
 'business': 11,
 'learn': 12,
 'view': 13,
 'use': 14,
 'ncert': 15,
 'medical': 16,
 'services': 17,
 'get': 18,
 'care': 19,
 'read': 20,
 'rebath': 21,
 'courses': 22,
 'social': 23,
 'see': 24,
 'help': 25,
 '2022': 26,
 'topics': 27,
 'information': 28,
 'learning': 29,
 'find': 30,
 'policy': 31,
 'online': 32,
 '₹': 33,
 'news': 34,
 'work': 35,
 'support': 36,
 'team': 37,
 'design': 38,
 'content': 39,
 'project': 40,
 'contact': 41,
 'privacy': 42,
 '10': 43,
 'best': 44,
 '2': 45,
 '1': 46,
 'projects': 47,
 'prof': 48,
 'instructorss': 49,
 'search': 50,
 'experience': 51,
 'jama': 52,
 'management': 53,
 'explore': 54,
 'page': 55,
 'resources': 56,
 'one': 57,
 'technology': 58,
 'medicine': 59,
 'education': 60,
 'construction': 61,
 'build': 62,
 'products': 63,
 'case': 64,
 'sign': 65,
 'need': 66,
 'make': 67,
 'healthcare': 68,
 'site': 69

In [68]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [69]:
print(train_sentences[10:15])
print(train_sequences[10:15])

['website stores cookies computer cookies used collect information interact website allow us remember use information order improve customize browsing experience analytics metrics visitors website media find cookies use see privacy policy accept find home back home types signature home custom home rentals view brokers find home home types signature home custom home rentals view brokers contact contact search perfect home design build custom home luxurious homes designed reflect way live entertain joe farrell previous featured properties south highway 54 miankoma lane amagansett 5995000 view featured properties modern barn 1264 townline road sagaponack 3995000 view featured properties transitional masterpiece 344 old sag harbor road bridgehampton 3995000 view next four types farrell homes building options designed suit needs explore ground 01 custom build explore crafted care 02 signature homes explore flexible approach 03 build suit explore discover life 04 selected rentals featured ar

In [70]:
# Pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Max number of words in a sequence
max_length = 1000

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape

((66, 1000), (17, 1000))

In [71]:
train_padded[10]

array([ 112, 6958,   92,   74,   92,  308, 1738,   28, 6959,  112, 1006,
          4, 3174,   14,   28,  236,  365, 1987, 2365,   51,  207, 2472,
       2958,  112,   84,   30,   92,   14,   24,   42,   31, 1249,   30,
          5,  282,    5, 1555, 1171,    5,  230,    5, 1302,   13, 4171,
         30,    5,    5, 1555, 1171,    5,  230,    5, 1302,   13, 4171,
         41,   41,   50, 1578,    5,   38,   62,  230,    5, 2473,  264,
       1172, 3175,  108,  336, 6960, 3880, 2474,  328,  126,  835,  466,
       4172, 4173, 6961, 3168, 6962, 6963,   13,  126,  835,  550, 6964,
       6965, 6966, 2475, 6967, 4174,   13,  126,  835, 1827, 4175, 6968,
       1173, 6969, 6970, 2475, 6971, 4174,   13,  118, 1440, 1555, 2474,
        264,  162,  406, 1172, 3176,  344,   54, 2110, 4176,  230,   62,
         54, 2476,   19, 3073, 1171,  264,   54, 3177,  609, 2421,   62,
       3176,   54,  227,  132, 4177, 1595, 1302,  126,  231,  162,  843,
       1174,  610, 3817,  485,   20,   13, 1441,  4

In [72]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

website stores cookies computer cookies used collect information interact website allow us remember use information order improve customize browsing experience analytics metrics visitors website media find cookies use see privacy policy accept find home back home types signature home custom home rentals view brokers find home home types signature home custom home rentals view brokers contact contact search perfect home design build custom home luxurious homes designed reflect way live entertain joe farrell previous featured properties south highway 54 miankoma lane amagansett 5995000 view featured properties modern barn 1264 townline road sagaponack 3995000 view featured properties transitional masterpiece 344 old sag harbor road bridgehampton 3995000 view next four types farrell homes building options designed suit needs explore ground 01 custom build explore crafted care 02 signature homes explore flexible approach 03 build suit explore discover life 04 selected rentals featured arti

In [73]:
# Check reversing the indices

# flip (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [74]:
reverse_word_index

{1: 'health',
 2: 'solutions',
 3: 'class',
 4: 'us',
 5: 'home',
 6: 'new',
 7: 'science',
 8: 'free',
 9: 'may',
 10: 'data',
 11: 'business',
 12: 'learn',
 13: 'view',
 14: 'use',
 15: 'ncert',
 16: 'medical',
 17: 'services',
 18: 'get',
 19: 'care',
 20: 'read',
 21: 'rebath',
 22: 'courses',
 23: 'social',
 24: 'see',
 25: 'help',
 26: '2022',
 27: 'topics',
 28: 'information',
 29: 'learning',
 30: 'find',
 31: 'policy',
 32: 'online',
 33: '₹',
 34: 'news',
 35: 'work',
 36: 'support',
 37: 'team',
 38: 'design',
 39: 'content',
 40: 'project',
 41: 'contact',
 42: 'privacy',
 43: '10',
 44: 'best',
 45: '2',
 46: '1',
 47: 'projects',
 48: 'prof',
 49: 'instructorss',
 50: 'search',
 51: 'experience',
 52: 'jama',
 53: 'management',
 54: 'explore',
 55: 'page',
 56: 'resources',
 57: 'one',
 58: 'technology',
 59: 'medicine',
 60: 'education',
 61: 'construction',
 62: 'build',
 63: 'products',
 64: 'case',
 65: 'sign',
 66: 'need',
 67: 'make',
 68: 'healthcare',
 69: 'site'

In [75]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [76]:
decoded_text = decode(train_sequences[10])

print(train_sequences[10])
print(decoded_text)

[112, 6958, 92, 74, 92, 308, 1738, 28, 6959, 112, 1006, 4, 3174, 14, 28, 236, 365, 1987, 2365, 51, 207, 2472, 2958, 112, 84, 30, 92, 14, 24, 42, 31, 1249, 30, 5, 282, 5, 1555, 1171, 5, 230, 5, 1302, 13, 4171, 30, 5, 5, 1555, 1171, 5, 230, 5, 1302, 13, 4171, 41, 41, 50, 1578, 5, 38, 62, 230, 5, 2473, 264, 1172, 3175, 108, 336, 6960, 3880, 2474, 328, 126, 835, 466, 4172, 4173, 6961, 3168, 6962, 6963, 13, 126, 835, 550, 6964, 6965, 6966, 2475, 6967, 4174, 13, 126, 835, 1827, 4175, 6968, 1173, 6969, 6970, 2475, 6971, 4174, 13, 118, 1440, 1555, 2474, 264, 162, 406, 1172, 3176, 344, 54, 2110, 4176, 230, 62, 54, 2476, 19, 3073, 1171, 264, 54, 3177, 609, 2421, 62, 3176, 54, 227, 132, 4177, 1595, 1302, 126, 231, 162, 843, 1174, 610, 3817, 485, 20, 13, 1441, 485, 5, 250, 418, 6972, 2474, 835, 6973, 6, 689, 403, 759, 4178, 13, 182, 264, 1441, 485, 5, 250, 651, 747, 42, 31, 4178, 115, 710, 2474, 162, 180, 186]
website stores cookies computer cookies used collect information interact website allow 

In [77]:
# Create LSTM model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

# Word embeddings give us a way to use an efficient, dense representation in which similar words have 
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify).

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.


model.add(layers.LSTM(256, dropout=0.1))
model.add(layers.Dense(1, activation="relu"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 32)          406816    
                                                                 
 lstm_1 (LSTM)               (None, 256)               295936    
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 703,009
Trainable params: 703,009
Non-trainable params: 0
_________________________________________________________________


In [86]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(lr=0.1)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [87]:
model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=2)

Epoch 1/20
3/3 - 64s - loss: 0.0000e+00 - accuracy: 0.2273 - val_loss: 0.0000e+00 - val_accuracy: 0.1176 - 64s/epoch - 21s/step
Epoch 2/20
3/3 - 8s - loss: 0.0000e+00 - accuracy: 0.2273 - val_loss: 0.0000e+00 - val_accuracy: 0.1176 - 8s/epoch - 3s/step
Epoch 3/20
3/3 - 8s - loss: 0.0000e+00 - accuracy: 0.2273 - val_loss: 0.0000e+00 - val_accuracy: 0.1176 - 8s/epoch - 3s/step
Epoch 4/20


KeyboardInterrupt: ignored

In [69]:
predictions = model.predict(train_padded)
predictions = [1 if p > 0.5 else 0 for p in predictions]

In [83]:
print(train_sentences[10:20])

print(train_labels[10:20])
print(predictions[10:20])

['watch demo try free use arrows select result press enter go selected search result touch device users use touch swipe gestures product platform domo customers analysts agree domo different business cloud learn domo’s platform technology integrate explore domo’s data integration capabilities visualize learn domo’s visualization technology build data apps create apps domo’s app framework leverage cloud use domo’s cloud use existing ones security explore domo’s robust security features governance centralize data governance domo product reporting dashboards create interactive custom dashboards ease selfservice analytics make data accessible free analysts’ time data sharing embedded analytics share analytics securely customers partners data apps provide teams data apps solve business problems product tour take quick interactive tour domo see products › enterprise domo enterprise manage massive data scale data integration worldclass governance data apps execute datadriven processes use cas

NameError: ignored

In [76]:
# from turtle import ht
from bs4 import BeautifulSoup
from bs4.element import Comment
import csv
import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()

# prepare urls
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)


urls = 'hhs.gov drugs.com epocrates.com everydayhealth.com healthline.com mayoclinic.org medicinenet.com medlineplus.gov medpagetoday.com medscape.com merckmanuals.com nih.gov openmd.com rxlist.com cdc.gov uptodate.com webmd.com biomedcentral.com freemedicaljournals.com jamanetwork.com nejm.org ncbi.nlm.nih.gov 1mg.com pharmeasy.in'.split(
    ' ')
# print(urls) #ctl00_ContentPlaceHolder1_DataList1_ctl00_StudentNameLabel


async def fetch(session, url):
    try:
        async with session.get(url) as response:
            text = await response.text()
            return text, url
    except Exception as e:
        print(str(e))


ldict = []


async def main():
    tasks = []

    async with aiohttp.ClientSession(headers=hdr, trust_env=True) as session:
        for url in urls:
            tasks.append(fetch(session=session, url=f'http://{url}'))
        htmls = await asyncio.gather(*tasks)
        for html in htmls:
            if html is not None:
                url = html[1]
                mdict = {}
                try:
                    mdict["url"] = url
                    mdict["type"] = 'medical'
                    mdict["text"] = text_from_html(html[0])
                    ldict.append(mdict)
                except Exception as e:
                    continue
# asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
asyncio.run(main())
with open('data.csv', 'w', encoding="utf-8") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=["url", "type", "text"])
    writer.writeheader()
    writer.writerows(ldict)


Server disconnected
Cannot connect to host nih.gov:80 ssl:default [Connect call failed ('156.40.212.210', 80)]


In [78]:
!du data.csv

188	data.csv
